## Setup

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_gan as tfgan
import numpy as np
import os, sys
from tqdm.notebook import tqdm
from pathlib import Path

sys.path.append( os.path.abspath('..') )
import utils

In [ ]:
Path('Fashion').mkdir(exist_ok=True)
os.chdir('Fashion')

In [ ]:
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()
data = np.concatenate((x_train, x_test))
data = (data.astype('float32') - 127.5) / 127.5  # normalize to [-1, 1]
data = np.expand_dims(data, axis=-1)  # add channels dimension

assert data.shape == (70000, 28, 28, 1)  # (batch, height, width, channel)

## 1 Models

### 1.1 Architecture

In [ ]:
def generator_model_transp_conv(latent_dims):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(7*7*256, input_shape=(latent_dims,)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Reshape((7, 7, 256)),

        tf.keras.layers.Conv2D(128, kernel_size=5, strides=1, padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding='same', activation='tanh')
    ])

In [ ]:
def generator_model_upsample(latent_dims, interpolation):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(7*7*256, input_shape=(latent_dims,)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Reshape((7, 7, 256)),
    
        tf.keras.layers.Conv2D(128, kernel_size=5, strides=1, padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.UpSampling2D(size=2, interpolation=interpolation),
        tf.keras.layers.Conv2D(64, kernel_size=5, strides=1, padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.UpSampling2D(size=2, interpolation=interpolation),
        tf.keras.layers.Conv2D(1, kernel_size=5, strides=1, padding='same', activation='tanh')
    ])

In [ ]:
def discriminator_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(28,28,1)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(128, kernel_size=5, strides=2, padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])

### 1.2 Losses

The binary cross entropy (BCE) between $y$ and $\hat{y}$ is calculated as:

$$
    \mathrm{BCE}(y, \hat{y}) = - y \log\left(\hat{y}\right) - (1-y) \log\left(1 - \hat{y}\right)
$$

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

The generator tries to maximize the chance of the discriminator being wrong. This is equivalent of trying to minimize the following loss function:

$$
    J^{(G)} = -\log\bigl(D\bigl(G(z)\bigr)\bigr)
$$

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

The discriminator tries to correctly classify real data as real and fake data as fake. This is equivalent to minimizing the following loss function:

$$
    J^{(D)} = -\log\bigr(D(x)\bigl) - \log\bigl(1 - D\bigl(G(z)\bigr)\bigr)
$$

Here we scale down the loss by a factor of $\;0.5$

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return 0.5 * (real_loss + fake_loss)

## 2 Training

### 2.1 Main functions

In [ ]:
def discriminator_train_step(generator, discriminator, images, latent_dims):
    noise = tf.random.normal([images.shape[0], latent_dims])
    with tf.GradientTape() as disc_tape:
        generated_imgs = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_imgs, training=True)
        loss_D = discriminator_loss(real_output, fake_output)
    
    grads_D = disc_tape.gradient(loss_D, discriminator.trainable_variables)
    discriminator.optimizer.apply_gradients(zip(grads_D, discriminator.trainable_variables))

In [ ]:
def generator_train_step(generator, discriminator, batch_size, latent_dims):
    noise = tf.random.normal([batch_size, latent_dims])
    with tf.GradientTape() as gen_tape:
        generated_imgs = generator(noise, training=True)
        fake_output = discriminator(generated_imgs, training=True)
        loss_G = generator_loss(fake_output)
    
    grads_G = gen_tape.gradient(loss_G, generator.trainable_variables)
    generator.optimizer.apply_gradients(zip(grads_G, generator.trainable_variables))

In [ ]:
def train(generator, discriminator, data, epochs, batch_size=32, callbacks=None):
    latent_dims = generator.input_shape[1]
    num_batches = 1 + (data.shape[0] - 1) // batch_size
    dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(data.shape[0]).batch(batch_size)
    
    generator_step = tf.function(generator_train_step)
    discriminator_step = tf.function(discriminator_train_step)
    for epoch in tqdm(range(epochs)):
        for c in callbacks:
            c.on_epoch_begin(epoch=epoch + 1, generator=generator, discriminator=discriminator)
        
        for batch in tqdm(dataset, leave=False, total=num_batches):
            discriminator_step(generator, discriminator, batch, latent_dims)
            generator_step(generator, discriminator, batch_size, latent_dims)
        
        for c in callbacks:
            c.on_epoch_end(epoch=epoch + 1, generator=generator, discriminator=discriminator)

### 2.2 Metrics classifier

Loading the classifier that will be used to calculate the *Classifier Score* (CS) and *Fréchet Classifier Distance* (FCD). \
The features of the real data are also precalculated to avoid doing that for each epoch.

In [ ]:
classifier = tf.keras.models.load_model('../../Classifiers/fashion.h5')
feature_layer = classifier.get_layer('features')
logits_layer = classifier.get_layer('logits')
precalculated_features = utils.fn.calculate_features(classifier, feature_layer, data)

### 2.3 Hyperparameter Testing

These were the hyperparameters tested for the final document. Training all of them simultaneously may take a long time, consider commenting out some options to run the tests individually.

In [ ]:
hparams_list = [
    {'batch_size': 16, 'latent_dims': 16, 'upsample': 'TrpConv'},
    {'batch_size': 16, 'latent_dims': 32, 'upsample': 'TrpConv'},
    {'batch_size': 32, 'latent_dims': 16, 'upsample': 'TrpConv'},
    {'batch_size': 32, 'latent_dims': 32, 'upsample': 'TrpConv'},
    
    {'batch_size':  8, 'latent_dims': 32, 'upsample': 'nearest'},
    {'batch_size': 16, 'latent_dims': 32, 'upsample': 'nearest'},
    {'batch_size': 32, 'latent_dims': 32, 'upsample': 'nearest'},
    
    {'batch_size':  8, 'latent_dims': 32, 'upsample': 'bilinear'},
    {'batch_size': 16, 'latent_dims': 16, 'upsample': 'bilinear'},
    {'batch_size': 16, 'latent_dims': 32, 'upsample': 'bilinear'},
    {'batch_size': 32, 'latent_dims': 16, 'upsample': 'bilinear'},
    {'batch_size': 32, 'latent_dims': 32, 'upsample': 'bilinear'},
    {'batch_size': 64, 'latent_dims': 32, 'upsample': 'bilinear'}
]

In [ ]:
for hparams in hparams_list:
    dirname = 'BS{}_LDIM{}_{}'.format(
        hparams['batch_size'],
        hparams['latent_dims'],
        hparams['upsample'].upper()
    )
    Path(dirname).mkdir(exist_ok=True)
    
    ## Models
    if hparams['upsample'] is 'TrpConv':
        generator = generator_model_transp_conv(hparams['latent_dims'])
    else:
        generator = generator_model_upsample(hparams['latent_dims'], hparams['upsample'])
    generator.optimizer = tf.keras.optimizers.Adam(1e-4)

    discriminator = discriminator_model()
    discriminator.optimizer = tf.keras.optimizers.Adam(1e-4)
    
    ## Callbacks
    timer = utils.callback.TimerCallback()
    save_samples = utils.callback.SaveSamplesCallback(
        path_format=os.path.join(dirname, 'epoch-{}'),
        inputs=tf.random.normal((10*10, hparams['latent_dims'])),
        n_cols=10,
        savefig_kwargs={'bbox_inches': 'tight', 'pad_inches': 0, 'dpi': 192},
        imshow_kwargs={'cmap': 'gray_r', 'vmin': -1, 'vmax': 1}
    )
    metrics = utils.callback.MetricsCallback(
        generator=generator,
        classifier=classifier,
        latent_dims=hparams['latent_dims'],
        feature_layer=feature_layer,
        logits_layer=logits_layer,
        precalculated_features=precalculated_features,
        save_after=5, save_to=os.path.join(dirname, 'best.h5')
    )
    
    ## Train and save results
    train(
        generator, discriminator, data, epochs=30,
        batch_size=hparams['batch_size'],
        callbacks=[timer, save_samples, metrics]
    )
    
    metrics_obj = metrics.get_metrics()
    metrics_obj['time'] = timer.get_time()
    utils.fn.update_json_log(os.path.join(dirname, 'log.json'), metrics_obj)
    
    generator.save    (os.path.join(dirname, 'generator.h5'    ), overwrite=True, save_format='h5')
    discriminator.save(os.path.join(dirname, 'discriminator.h5'), overwrite=True, save_format='h5')

\
In windows the command bellow is used to turn down the machine after the training finishes, very useful if you wanna let the computer running while you go to sleep :)

In [ ]:
# !shutdown /s /t 60